In [1]:
import pandas as pd
import numpy as np

In [2]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:  # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2  # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [3]:
csv_opts = {'sep': '|',
           'quotechar': '"',
           'compression': 'gzip',
           'encoding': 'utf-8',
           'header': 5}

In [4]:
arrest_dtypes = {'Area of Responsibility': 'category',
 'Apprehension Date': 'str',
 'Apprehension Method': 'category',
 'Apprehension Landmark': 'category',
 'Operation': 'category',
 'Processing Disposition': 'category',
 'Birth Date': 'category',
 'Citizenship': 'category',
 'Gender': 'category'}

encounter_dtypes = {'Event Area of Responsibility': 'category',
 'Event Date': 'str',
 'Landmark': 'category',
 'Operation': 'category',
 'Processing Disposition': 'category',
 'Birth Date': 'category',
 'Citizenship': 'category',
 'Gender': 'category'}

removal_dtypes = {'Area of Responsibility': 'category',
 'Apprehension Date': 'str',
 'Processing Disposition Code': 'category',
 'Birth Date': 'category',
 'Citizenship': 'category',
 'Gender': 'category',
 'RC Threat Level': 'category',
 'Final Charge Section': 'category',
 'Departed Date': 'str',
 'Removal Date': 'str'}

In [5]:
arrests16 = pd.read_csv('../input/EROArrests_FY2016.csv.gz', 
                        **csv_opts,
                        dtype=arrest_dtypes,
                        parse_dates=['Apprehension Date'])
arrests17 = pd.read_csv('../input/EROArrests_FY2017.csv.gz', 
                        **csv_opts,
                        dtype=arrest_dtypes, 
                        parse_dates=['Apprehension Date'])
arrests18 = pd.read_csv('../input/EROArrests_FY2018.csv.gz', 
                        **csv_opts,
                        dtype=arrest_dtypes, 
                        parse_dates=['Apprehension Date'])
arrests19 = pd.read_csv('../input/EROArrests_FY2019.csv.gz', 
                        **csv_opts,
                        dtype=arrest_dtypes, 
                        parse_dates=['Apprehension Date'])
encounters16 = pd.read_csv('../input/EROEncounters_FY2016.csv.gz',
                           **csv_opts,
                           dtype=encounter_dtypes,
                           parse_dates=['Event Date'])
encounters17 = pd.read_csv('../input/EROEncounters_FY2017.csv.gz',
                           **csv_opts,
                           dtype=encounter_dtypes,
                           parse_dates=['Event Date'])
encounters18 = pd.read_csv('../input/EROEncounters_FY2018.csv.gz',
                           **csv_opts,
                           dtype=encounter_dtypes,
                           parse_dates=['Event Date'])
encounters19 = pd.read_csv('../input/EROEncounters_FY2019.csv.gz',
                           **csv_opts,
                           dtype=encounter_dtypes,
                           parse_dates=['Event Date'])
removals16 = pd.read_csv('../input/ICERemovals_FY2016.csv.gz',
                         **csv_opts,
                        dtype=removal_dtypes,
                        parse_dates=['Departed Date', 'Removal Date'])
removals17 = pd.read_csv('../input/ICERemovals_FY2017.csv.gz',
                         **csv_opts,
                        dtype=removal_dtypes,
                        parse_dates=['Departed Date', 'Removal Date'])
removals18 = pd.read_csv('../input/ICERemovals_FY2018.csv.gz',
                         **csv_opts,
                        dtype=removal_dtypes,
                        parse_dates=['Departed Date', 'Removal Date'])
removals19 = pd.read_csv('../input/ICERemovals_FY2019.csv.gz',
                         **csv_opts,
                        dtype=removal_dtypes,
                        parse_dates=['Departed Date', 'Removal Date'])

In [6]:
arrests = pd.concat([arrests16, arrests17, arrests18, arrests19])
encounters = pd.concat([encounters16, encounters17, encounters18, encounters19])
removals = pd.concat([removals16, removals17, removals18, removals19])

In [7]:
# Convert 'object' columns to categories, where efficient.
# Implementation via https://www.dataquest.io/blog/pandas-big-data/
dfs_to_convert = [arrests, encounters, removals]
for df in dfs_to_convert:
    df_obj = df.select_dtypes(include=['object']).copy()
    converted_obj = pd.DataFrame()

    for col in df_obj.columns:
        num_unique_values = len(df_obj[col].unique())
        num_total_values = len(df_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:, col] = df_obj[col].astype('category')
        else:
            converted_obj.loc[:, col] = df_obj[col]

    df[converted_obj.columns] = converted_obj
    del df_obj, converted_obj

In [8]:
arrests.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544059 entries, 0 to 131903
Data columns (total 9 columns):
Area of Responsibility    540251 non-null category
Apprehension Date         544059 non-null datetime64[ns]
Apprehension Method       544059 non-null category
Apprehension Landmark     529216 non-null category
Operation                 117368 non-null category
Processing Disposition    543865 non-null category
Birth Date                544059 non-null category
Citizenship               544059 non-null category
Gender                    544059 non-null category
dtypes: category(8), datetime64[ns](1)
memory usage: 13.9 MB


In [9]:
encounters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1689378 entries, 0 to 396383
Data columns (total 8 columns):
Event Area of Responsibility    1608957 non-null category
Event Date                      1689378 non-null datetime64[ns]
Landmark                        1029394 non-null category
Operation                       298737 non-null category
Processing Disposition          1686035 non-null category
Birth Date                      1689378 non-null category
Citizenship                     1689378 non-null category
Gender                          1689360 non-null category
dtypes: category(7), datetime64[ns](1)
memory usage: 40.7 MB


In [10]:
removals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 963972 entries, 0 to 241512
Data columns (total 10 columns):
Area of Responsibility         963972 non-null category
Apprehension Date              949737 non-null category
Processing Disposition Code    958257 non-null category
Birth Date                     963972 non-null category
Citizenship                    963972 non-null category
Gender                         963972 non-null category
RC Threat Level                542478 non-null category
Final Charge Section           962328 non-null category
Departed Date                  963972 non-null datetime64[ns]
Removal Date                   963972 non-null datetime64[ns]
dtypes: category(8), datetime64[ns](2)
memory usage: 31.7 MB


In [11]:
individual_dfs = [arrests16,
                 arrests17,
                 arrests18,
                 arrests19,
                 encounters16,
                 encounters17,
                 encounters18,
                 encounters19,
                 removals16,
                 removals17,
                 removals18,
                 removals19]
for df in individual_dfs:
    print(len(df))
    del df

110104
143470
158581
131904
368645
439903
484446
396384
240255
226119
256085
241513


In [12]:
del arrests16, arrests17, arrests18, arrests19, encounters16, encounters17, encounters18, encounters19, removals16, removals17, removals18, removals19

In [13]:
redacted = ['Birth Date']
arrests = arrests.drop(redacted, axis=1)
arrests.columns = arrests.columns.str.lower()
arrests.columns = arrests.columns.str.replace(' ', '_')

In [14]:
redacted = ['Birth Date']
encounters = encounters.drop(redacted, axis=1)
encounters.columns = encounters.columns.str.lower()
encounters.columns = encounters.columns.str.replace(' ', '_')

In [15]:
redacted = ['Birth Date']
removals = removals.drop(redacted, axis=1)
removals.columns = removals.columns.str.lower()
removals.columns = removals.columns.str.replace(' ', '_')

In [16]:
arrests['id'] = range(len(arrests))
removals['id'] = range(len(removals))
encounters['id'] = range(len(encounters))

In [17]:
arrests = arrests.rename({'area_of_responsibility': 'aor'}, axis=1)
encounters = encounters.rename({'event_area_of_responsibility': 'aor'}, axis=1)
removals = removals.rename({'area_of_responsibility': 'aor'}, axis=1)

In [18]:
removals.aor = removals.aor.str.replace('Area of Responsibility', '')
removals.aor = removals.aor.astype('category')
arrests.aor = arrests.aor.str.replace('Area of Responsibility', '')
arrests.aor = arrests.aor.astype('category')
encounters.aor = encounters.aor.str.replace('Area of Responsibility', '')
encounters.aor = encounters.aor.astype('category')

In [19]:
arrests.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544059 entries, 0 to 131903
Data columns (total 9 columns):
aor                       540251 non-null category
apprehension_date         544059 non-null datetime64[ns]
apprehension_method       544059 non-null category
apprehension_landmark     529216 non-null category
operation                 117368 non-null category
processing_disposition    543865 non-null category
citizenship               544059 non-null category
gender                    544059 non-null category
id                        544059 non-null int64
dtypes: category(7), datetime64[ns](1), int64(1)
memory usage: 17.5 MB


In [20]:
grouper = [pd.Grouper(freq='AS-OCT'), 'aor']
arrests.set_index('apprehension_date').groupby(grouper)['id'].count()

apprehension_date  aor            
2015-10-01         Atlanta             8866
                   Baltimore           1239
                   Boston              1858
                   Buffalo             1173
                   Chicago             7055
                   Dallas              9634
                   Denver              2284
                   Detroit             2241
                   El Paso             1611
                   HQ                    12
                   Houston            12896
                   Los Angeles         7651
                   Miami               3524
                   New Orleans         5174
                   New York City       1847
                   Newark              2247
                   Philadelphia        3672
                   Phoenix             5370
                   Salt Lake City      4638
                   San Antonio         8425
                   San Diego           3683
                   San Francisco       66